In [1]:
import pandas as pd
import numpy as np
import glob as gl
import pickle
from efficient_apriori import apriori
import warnings
import os
import pathlib
warnings.filterwarnings('ignore')

In [2]:
sub_MR1 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSub_MR1.csv', index_col= 0)
sub_MR2 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSub_MR2.csv', index_col= 0)
sub_MR3 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSub_MR3.csv', index_col= 0)
sub_MR4 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSub_MR4.csv', index_col= 0)

In [3]:
sum_MR1 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSum_MR1.csv', index_col= 0)
sum_MR2 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSum_MR2.csv', index_col= 0)
sum_MR3 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSum_MR3.csv', index_col= 0)
sum_MR4 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSum_MR4.csv', index_col= 0)

In [4]:
def alejaEncoding(df):

    data = df.copy()

    for index, row in df.iterrows():
        a = row['input_a']
        b = row['input_b']

        if a > b and b != 0:
            data.at[index, 'encoding'] =  'a > b'
        if a < b and a != 0:
            data.at[index, 'encoding'] =  'a < b'
        if a == b :
            data.at[index, 'encoding'] =  'a = b'
        
        if a == b == 0 :
            data.at[index, 'encoding'] =  'a == b == 0'
        
        if a > b and b == 0 :
            data.at[index, 'encoding'] =  'a > b (b == 0)'
        
        if a < b and a == 0 :
            data.at[index, 'encoding'] =  'a < b (a == 0)'

    return data
            
        

In [5]:
def ruleGenerator(transactions_df, support, conf):
    """
        Generate set of rules with given support and confidence using transactions_df
    """
    # print(transactions_df.head(20))
    transactions_array = transactions_df.to_numpy()
    transactions_list = transactions_array.tolist()
    itemSets, rules = apriori(transactions_list, min_support=support, min_confidence=conf)
    return itemSets, rules


In [6]:
def ruleGen(items, rules1):
    sets1 = []
    sets2 = []
    dfListUnion = []
    dfListIntersection = []
    sets1.append(set(rules1))
    sets2.append(set(items))

    union = set.union(*sets1)
    lUnion = list(union)

    for rul in range(0, len(lUnion)):
                dfAux_union = {'LHS': tuple(lUnion[rul].lhs), 'LHS_size': len(lUnion[rul].lhs),
                            'RHS': tuple(lUnion[rul].rhs), 'RHS_size': len(lUnion[rul].rhs),
                            'conf': lUnion[rul].confidence,
                            'sup': lUnion[rul].support, 'lift': lUnion[rul].lift, 'cov': lUnion[rul].conviction}
                dfListUnion.append(dfAux_union)

    dfRule = pd.DataFrame(dfListUnion)

    return dfRule

## SUB

In [7]:
sub_MR1_encode = alejaEncoding(sub_MR1)
sub_MR2_encode = alejaEncoding(sub_MR2)
sub_MR3_encode = alejaEncoding(sub_MR3)
sub_MR4_encode = alejaEncoding(sub_MR4)

In [8]:
sub_MR1_encode

,"(a,b)",input_a,input_b,operand,const,output,MR1_input_a,MR1_input_b,MR1_inputs,MR1_output,MR1_checker,encoding
0,"(9,2)",9,2,sub,6,7.0,2.0,9.0,"(2,9)",-7.0,Violated,a > b
1,"(8,1)",8,1,sub,6,7.0,1.0,8.0,"(1,8)",-7.0,Violated,a > b
2,"(4,8)",4,8,sub,6,-4.0,8.0,4.0,"(8,4)",4.0,Violated,a < b
3,"(7,2)",7,2,sub,6,5.0,2.0,7.0,"(2,7)",-5.0,Violated,a > b
4,"(0,6)",0,6,sub,6,-6.0,6.0,0.0,"(6,0)",6.0,Violated,a < b (a == 0)
...,...,...,...,...,...,...,...,...,...,...,...,...
95,"(2,7)",2,7,sub,6,-5.0,7.0,2.0,"(7,2)",5.0,Violated,a < b
96,"(2,3)",2,3,sub,6,-1.0,3.0,2.0,"(3,2)",1.0,Violated,a < b
97,"(1,3)",1,3,sub,6,-2.0,3.0,1.0,"(3,1)",2.0,Violated,a < b
98,"(0,6)",0,6,sub,6,-6.0,6.0,0.0,"(6,0)",6.0,Violated,a < b (a == 0)


In [9]:
sub_MR1_encode_1feature = sub_MR1_encode[['encoding','MR1_checker']]
sub_MR2_encode_1feature = sub_MR2_encode[['encoding','MR2_checker']]
sub_MR3_encode_1feature = sub_MR3_encode[['encoding','MR3_checker']]
sub_MR4_encode_1feature = sub_MR4_encode[['encoding','MR4_checker']]

In [10]:
items_MR1_sub, rules1_MR1_sub = ruleGenerator(sub_MR1_encode_1feature, 0.01, float(1))
items_MR2_sub, rules1_MR2_sub = ruleGenerator(sub_MR2_encode_1feature, 0.01, float(1))
items_MR3_sub, rules1_MR3_sub = ruleGenerator(sub_MR3_encode_1feature, 0.01, float(1))
items_MR4_sub, rules1_MR4_sub = ruleGenerator(sub_MR4_encode_1feature, 0.01, float(1))

In [11]:
dfRule_MR1_sub = ruleGen(items_MR1_sub, rules1_MR1_sub)
dfRule_MR2_sub = ruleGen(items_MR2_sub, rules1_MR2_sub)
dfRule_MR3_sub = ruleGen(items_MR3_sub, rules1_MR3_sub)
dfRule_MR4_sub = ruleGen(items_MR4_sub, rules1_MR4_sub)

In [12]:
dfRule_MR1_sub.to_csv('rules_sub_MR1_RI.csv')
dfRule_MR2_sub.to_csv('rules_sub_MR2_RI.csv')
dfRule_MR3_sub.to_csv('rules_sub_MR3_RI.csv')
dfRule_MR4_sub.to_csv('rules_sub_MR4_RI.csv')

## SUM/ADD

In [13]:
sum_MR1_encode = alejaEncoding(sum_MR1)
sum_MR2_encode = alejaEncoding(sum_MR2)
sum_MR3_encode = alejaEncoding(sum_MR3)
sum_MR4_encode = alejaEncoding(sum_MR4)

In [14]:
sum_MR1_encode_1feature = sum_MR1_encode[['encoding','MR1_checker']]
sum_MR2_encode_1feature = sum_MR2_encode[['encoding','MR2_checker']]
sum_MR3_encode_1feature = sum_MR3_encode[['encoding','MR3_checker']]
sum_MR4_encode_1feature = sum_MR4_encode[['encoding','MR4_checker']]

In [15]:
items_MR1_sum, rules1_MR1_sum = ruleGenerator(sum_MR1_encode_1feature, 0.01, float(1))
items_MR2_sum, rules1_MR2_sum = ruleGenerator(sum_MR2_encode_1feature, 0.01, float(1))
items_MR3_sum, rules1_MR3_sum = ruleGenerator(sum_MR3_encode_1feature, 0.01, float(1))
items_MR4_sum, rules1_MR4_sum = ruleGenerator(sum_MR4_encode_1feature, 0.01, float(1))

In [16]:
dfRule_MR1_sum = ruleGen(items_MR1_sum, rules1_MR1_sum)
dfRule_MR2_sum = ruleGen(items_MR2_sum, rules1_MR2_sum)
dfRule_MR3_sum = ruleGen(items_MR3_sum, rules1_MR3_sum)
dfRule_MR4_sum = ruleGen(items_MR4_sum, rules1_MR4_sum)

In [17]:
dfRule_MR1_sum.to_csv('rules_sum_MR1_RI.csv')
dfRule_MR2_sum.to_csv('rules_sum_MR2_RI.csv')
dfRule_MR3_sum.to_csv('rules_sum_MR3_RI.csv')
dfRule_MR4_sum.to_csv('rules_sum_MR4_RI.csv')